In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## This cell below is the starter code provided by Udacity.

In [ ]:
import sys
import pickle
sys.path.append("./tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
features_list = ['poi',
                 'total_stock_value',
                 'total_payments',
                 'restricted_stock',
                 'exercised_stock_options',
                 'salary',
                 'expenses',
                 'other',
                 'to_messages',
                 'shared_receipt_with_poi',
                 'from_messages',
                 'from_this_person_to_poi',
                 'from_poi_to_this_person',
                 'bonus']

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

### Task 2: Remove outliers

### Task 3: Create new feature(s)
### Store to my_dataset for easy export below.
my_dataset = data_dict

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

### Task 4: Try a variety of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

# Provided to give you a starting point. Try a variety of classifiers.
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

### Task 5: Tune your classifier to achieve better than .3 precision and recall
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info:
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

# I don't need to dump it right now.
# dump_classifier_and_data(clf, my_dataset, features_list)

In [ ]:
# Investigate amount of usable data for each data point
TOTAL_ITEMS = len(data_dict[data_dict.keys()[0]].keys()) - 1 # minus 1 to exclude POI indicator
proportion_unusable_data = {}
for person, data in data_dict.items():
    num_NaN = 0
    for key, value in data.items():
        if key != 'poi':
            if value == 'NaN':
                num_NaN += 1
    proportion_unusable_data[person] = num_NaN / float(TOTAL_ITEMS)

In [ ]:
sorted(proportion_unusable_data.items(), key=lambda x: x[1], reverse=True)

## Data Exploration

In [ ]:
# Let's take a look at our data set and get familiar with our data points
num_data_points = len(data_dict)

num_poi, num_no_poi = 0, 0
for person in data_dict.keys():
    if data_dict[person]['poi'] == 0:
        num_no_poi += 1
    else:
        num_poi += 1

print("Number of data points: " + str(num_data_points))
print("Number of POI: " + str(num_poi))
print("Number of non-POI: " + str(num_no_poi))
print("Proportion of POI to non-POI: " + str(num_poi/float(num_no_poi)))

# Count total features available "out of the box"
features_list = data_dict[data_dict.keys()[0]].keys()
print("Number of features available at the start: "
      + str(len(features_list) - 1))    # minus one to exclude 'poi' label
                                        # which is not a feature

# Are there features with many missing values?
missing_values = []

for feature in [f for f in features_list if f != 'poi']:
    num_nan = 0
    for person in data_dict.keys():
        feature_value = data_dict[person][feature]
        if feature_value == 'NaN':  # Missing values are encoded as 'NaN'
            num_nan += 1
    print("Feature '"+feature+"' has "+str(num_nan)+" missing values out of "
          +str(num_data_points)+", or "+str(num_nan/float(num_data_points))+"")
    missing_values.append((feature, num_nan, num_data_points, num_nan/float(num_data_points)))


# Prep report table
print("| Feature | # Missing Values | Proportion of Missing Values |") # header
print("|:--|:--|:--|") # formatting
# Sort list of features according to number of missing values:
for row in sorted(missing_values, key=lambda x: x[3]):
    print("| "+str(row[0])+" | "+str(row[1])+" | "+str(round(row[3], 3))+" |")

In [ ]:
clf.fit(features_train, labels_train)
predictions = clf.predict(features_test)
clf.score(features_test, labels_test)

In [ ]:
# Test for errors:
from sklearn.metrics import precision_score, recall_score

print('Precision: '+str(precision_score(labels_test, predictions)))
print('Recall: '+str(recall_score(labels_test, predictions)))

## Outlier Visualization

In [ ]:
def load_features(features_list):
    "Auxiliary function to load new features from data set."
    ### The first feature in features_list must be "poi".
    ### Extract features and labels from dataset for local testing
    data = featureFormat(my_dataset, features_list, sort_keys = True)
    labels, features = targetFeatureSplit(data)

    return labels, features

In [ ]:
def plot_features(features, labels, features_list):
    ### The first feature in features_list must be "poi".

    # Color POIs as red and non-POIs as blue
    color_map = map(lambda x: 'b' if x == 0 else 'r', labels)

    plt.scatter([f[0] for f in features],
               [f[1] for f in features],
               c=color_map)
    plt.xlabel(features_list[1])
    plt.ylabel(features_list[2])


def get_n_max_val(features, features_list, n=1):
    """Return n maximum values for features.
    The first feature in features_list must be "poi"."""
    
    max_val = []
    for i in range(len(features_list[1:])):
        values = [feat[i] for feat in features]
        max_val.append((features_list[i+1], sorted(values)[-n:]))
        
    return max_val

In [ ]:
list_of_features_lists = [
    ['poi','total_stock_value', 'total_payments'],
    ['poi', 'restricted_stock', 'exercised_stock_options'],
    ['poi', 'salary', 'expenses']
]

plt.figure(figsize=(6,7))

i = 1  # subplot index
for features_list in list_of_features_lists:
    labels, features = load_features(features_list)
    
    plt.subplot(3,1,i)
    plot_features(features, labels, features_list)
    
    i += 1

    # Print max value for current features
    print(get_n_max_val(features, features_list))

plt.tight_layout()
#plt.savefig('exploration_outliers.png', dpi=400)

In [ ]:
for person, values in data_dict.items():
    if values['salary'] == 26704229.0:
        print person

In [ ]:
sorted(data_dict.keys())

In [ ]:
# Remove TOTAL from our data set
data_dict.pop('TOTAL')

## Trying out RFE; didn't quite work

In [ ]:
# Let's try some RFE
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVC, LinearSVC

In [ ]:
clf = SVC(kernel='linear')
selector = RFECV(clf)
selector = selector.fit(features_train, labels_train)

print("Num Features: %d") % selector.n_features_
print("Selected Features: %s") % selector.support_
print("Feature Ranking: %s") % selector.ranking_

selector.score(features_test, labels_test)

## Trying out SelectPercentile + SelectKBest

In [ ]:
from sklearn.feature_selection import SelectPercentile, SelectKBest, f_classif
from sklearn.metrics import precision_score, recall_score
from sklearn.svm import SVC, LinearSVC

In [ ]:
# selector = SelectPercentile(f_classif, percentile=20)
selector = SelectKBest(f_classif, k=3)
selector.fit(features_train, labels_train)

sorted(zip(selector.scores_, features_list[1:]), reverse=True)

In [ ]:
# Using GaussianNB with ALL features
clf = GaussianNB()

clf.fit(features_train, labels_train)
predictions = clf.predict(features_test)
clf.score(features_test, labels_test)

In [ ]:
# Test for errors:
print('Precision: '+str(precision_score(labels_test, predictions)))
print('Recall: '+str(recall_score(labels_test, predictions)))

In [ ]:
# Using GaussianNB with SELECTED features
clf_selected = GaussianNB()

transformed_features_train = selector.transform(features_train)
transformed_features_test = selector.transform(features_test)

clf_selected.fit(transformed_features_train, labels_train)
pred_selected = clf_selected.predict(transformed_features_test)
clf_selected.score(transformed_features_test, labels_test)

In [ ]:
# Test for errors:
print('Precision: '+str(precision_score(labels_test, pred_selected)))
print('Recall: '+str(recall_score(labels_test, pred_selected)))

In [ ]:
# Trying Decision Trees + AdaBoost
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
n_estimators = 50
clf_selected = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=6, min_samples_leaf=1),
    n_estimators=n_estimators)

transformed_features_train = selector.transform(features_train)
transformed_features_test = selector.transform(features_test)

clf_selected.fit(transformed_features_train, labels_train)
pred_selected = clf_selected.predict(transformed_features_test)
clf_selected.score(transformed_features_test, labels_test)

# clf_selected.feature_importances_

In [ ]:
# Test for errors:
print('Precision: '+str(precision_score(labels_test, pred_selected)))
print('Recall: '+str(recall_score(labels_test, pred_selected)))

In [ ]:
n_estimators = 50
clf = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=6, min_samples_leaf=1),
    n_estimators=n_estimators)

clf.fit(features_train, labels_train)
predictions = clf.predict(features_test)
clf.score(features_test, labels_test)

In [ ]:
# Test for errors:
print('Precision: '+str(precision_score(labels_test, predictions)))
print('Recall: '+str(recall_score(labels_test, predictions)))

In [ ]:
# Let me try SVMs

# Feature Scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(features_train)

clf = SVC()
clf.fit(scaler.transform(features_train), labels_train)

predictions = clf.predict(scaler.transform(features_test))

clf.score(scaler.transform(features_test), labels_test)

In [ ]:
# Test for errors:
print('Precision: '+str(precision_score(labels_test, predictions)))
print('Recall: '+str(recall_score(labels_test, predictions)))

In [ ]:
clf_selected = SVC()

scaled_features_train = scaler.transform(features_train)
scaled_features_test = scaler.transform(features_test)

clf_selected.fit(selector.transform(scaled_features_train), labels_train)
predictions = clf_selected.predict(selector.transform(scaled_features_test))
clf_selected.score(selector.transform(scaled_features_test), labels_test)

In [ ]:
# Test for errors:
print('Precision: '+str(precision_score(labels_test, predictions)))
print('Recall: '+str(recall_score(labels_test, predictions)))

In [ ]:
# clf_selected = SVC(C=0.1)
# clf_selected = SVC(C=1)
# clf_selected = SVC(C=10)
# clf_selected = SVC(C=100)

clf_selected = SVC(gamma=0.01)
# clf_selected = SVC(gamma=1)
# clf_selected = SVC(gamma=10)
# clf_selected = SVC(gamma=100)

scaled_features_train = scaler.transform(features_train)
scaled_features_test = scaler.transform(features_test)

clf_selected.fit(selector.transform(scaled_features_train), labels_train)
predictions = clf_selected.predict(selector.transform(scaled_features_test))
clf_selected.score(selector.transform(scaled_features_test), labels_test)

### Trying GridSearchCV

In [ ]:
from sklearn.grid_search import GridSearchCV

In [ ]:
parameters = {
    'kernel':('linear', 'rbf'),
    'C':[1, 10, 100]
}

svc = SVC()
clf_selected = GridSearchCV(svc, parameters, scoring='f1')

scaled_features_train = scaler.transform(features_train)
scaled_features_test = scaler.transform(features_test)

clf_selected.fit(selector.transform(scaled_features_train), labels_train)
predictions = clf_selected.predict(selector.transform(scaled_features_test))
clf_selected.score(selector.transform(scaled_features_test), labels_test)

clf_selected.grid_scores_

### Trying out StratifiedShuffleSplit

In [ ]:
from sklearn.cross_validation import StratifiedShuffleSplit

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
iter_num = 0

precision_list = []
recall_list = []

for train_index, test_index in sss:
    features_train = []
    features_test  = []
    labels_train   = []
    labels_test    = []
    
    for ii in train_index:
        features_train.append( features[ii] )
        labels_train.append( labels[ii] )
    for jj in test_index:
        features_test.append( features[jj] )
        labels_test.append( labels[jj] )
        
    # Select features
    selector = SelectKBest(f_classif, k=3)
    selector.fit(features_train, labels_train)
    print('Features: '+str(sorted(zip(selector.scores_, features_list[1:]), reverse=True)[:3]))
    
    # Classifier
#     clf_selected = DecisionTreeClassifier(random_state=42)
    clf_selected = GaussianNB()

    selected_features_train = selector.transform(features_train)
    selected_features_test = selector.transform(features_test)

    clf_selected.fit(selected_features_train, labels_train)
    pred_selected = clf_selected.predict(selected_features_test)
    
    # Test for errors:
    precision = precision_score(labels_test, pred_selected)
    recall = recall_score(labels_test, pred_selected)
    precision_list.append(precision)
    recall_list.append(recall)
    
    print('Iteration: '+str(iter_num))
    print('Precision: '+str(precision))
    print('Recall: '+str(recall))
    print
    
    iter_num += 1

print('Average precison: '+str(np.mean(precision_list)))
print('Average recall: '+str(np.mean(recall_list)))

In [ ]:
#!/usr/bin/python

import sys
import pickle
sys.path.append("./tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# Initializing dataset for local testing
features_list = ['poi',
                 'total_stock_value',
                 'total_payments',
                 'restricted_stock',
                 'exercised_stock_options',
                 'salary',
                 'expenses',
                 'other',
                 'to_messages',
                 'shared_receipt_with_poi',
                 'from_messages',
                 'from_this_person_to_poi',
                 'from_poi_to_this_person',
                 'bonus']

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

data_dict.pop('TOTAL')

my_dataset = data_dict

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)


def test_classifier(clf, data_sets):
    iter_num = 0

    precision_list = []
    recall_list = []


    for data_set in data_sets:
        features_train, labels_train, features_test, labels_test = data_set

        # Select features
        selector = SelectKBest(f_classif, k=3)
        selector.fit(features_train, labels_train)
        print('Features: '+str(sorted(zip(selector.scores_, features_list[1:]), reverse=True)[:3]))

        selected_features_train = selector.transform(features_train)
        selected_features_test = selector.transform(features_test)

        # Fit classifier and make predictions
        clf.fit(selected_features_train, labels_train)
        pred_selected = clf.predict(selected_features_test)

        # Test for errors:
        precision = precision_score(labels_test, pred_selected)
        recall = recall_score(labels_test, pred_selected)
        precision_list.append(precision)
        recall_list.append(recall)

        print('Iteration: '+str(iter_num))
        print('Precision: '+str(precision))
        print('Recall: '+str(recall))
        print

        iter_num += 1

    print('classifier: '+str(clf))
    print('Average precison: '+str(np.mean(precision_list)))
    print('Average recall: '+str(np.mean(recall_list)))
    print


def split_data_sss(features, labels):
    '''Uses StratifiedShuffleSplit to create train and test data.
    Returns a list of tuples.'''
    sss = StratifiedShuffleSplit(y=labels, test_size=0.3, n_iter=10)

    train_test_sets = []

    for train_index, test_index in sss:
        features_train = []
        features_test  = []
        labels_train   = []
        labels_test    = []

        for i in train_index:
            features_train.append(features[i])
            labels_train.append(labels[i])
        for j in test_index:
            features_test.append(features[j])
            labels_test.append(labels[j])

        train_test_sets.append((features_train, labels_train, features_test, labels_test))

    return train_test_sets


# Test our classifiers
classifiers = [
    GaussianNB(),
    DecisionTreeClassifier(random_state=42)
]

# Split data into train/test data sets
data_sets = split_data_sss(features, labels)

for clf in classifiers:
    test_classifier(clf, data_sets)


In [ ]:
tup = (23.18661338478892, 'exercised_stock_options')
tup[1]

## New feature

In [ ]:
def create_proportion_from_poi(data_point):
    '''Creates a new feature: The proportion of all emails received by this person
    that were sent from a POI.'''

    FEATURE_NAME = 'proportion_from_poi'

    data_point[FEATURE_NAME] = float(data_point['from_poi_to_this_person'])/float(data_point['to_messages'])


def update_data_dict_proportion_from_poi(data_dict):
    '''Updates data_dict with new feature.'''

    for data in data_dict.values():
        create_proportion_from_poi(data)

In [ ]:
update_data_dict_proportion_from_poi(data_dict)

In [ ]:
proportion_from_poi_list = []
for person, data in data_dict.items():
    proportion_from_poi_list.append((person, data['proportion_from_poi']))
    
sorted(proportion_from_poi_list, key=lambda x: x[1], reverse=True)